# Experiment Analysis: MNIST-like

In [ ]:
import os
import tqdm
import json
import copy

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

Fully flatten the dictionary.

In [ ]:
from cplxpaper.auto.parameter_grid import get_params

def get_details(self):
    out = dict()
    for key in self:
        value = self[key]
        if isinstance(value, (dict, list, tuple)):
            if isinstance(value, (list, tuple)):
                value = {f"[{i}]": v for i, v in enumerate(value)}
                nested = get_details(value).items()
                out.update((key + k, val) for k, val in nested)

            elif isinstance(value, dict):
                nested = get_details(value).items()
                out.update((key + '__' + k, val) for k, val in nested)

            continue

        out[key] = value

    return out

Load performance results from each snapshot in the experiment.

In [ ]:
from cplxpaper.auto.utils import load_snapshot

def from_snapshots(*snapshots):
    results, options = {}, {}
    for snapshot in sorted(snapshots):
        name = os.path.basename(snapshot)
        snapshot = load_snapshot(snapshot)

        options = snapshot['options']
        stage, settings = snapshot['stage']

        results[name] = stage, snapshot['performance']

    return results, options

load experimnet from its snapshots or from cache

In [ ]:
import re
import pickle


def load_experiment(folder, cache="cache.pk"):
    if isinstance(cache, str):
        cache = os.path.join(folder, cache)

    assert cache is None or isinstance(cache, str)

    snapshots = []
    folder, _, filenames = next(os.walk(folder))
    for filename in sorted(filenames):
        if re.match(r"^\d+.*\.gz$", filename) is not None:
            snapshots.append(filename)

    # load scorer results from the snapshots or from cache
    scores, options = {}, {}
    if cache is not None and os.path.exists(cache):
        with open(cache, "rb") as fin:
            scores, options = pickle.load(fin)

    # reload from originals af anything is missing (use SHA-digest)
    if any(s not in scores for s in snapshots):
        snapshots = [os.path.join(folder, s) for s in snapshots]
        scores, options = from_snapshots(*snapshots)
        if cache is not None:
            with open(cache, "wb") as fout:
                pickle.dump((scores, options), fout)

    return scores, options

## Cache the results

In [ ]:
source = "./grids/minst-like/"

Collect results and reconstruct the grid

In [ ]:
import pandas as pd
from collections import defaultdict

grid = defaultdict(set)
ignore = {"__name__", "__timestamp__", "__version__", "device"}

results = []
source, experiments, manifests = next(os.walk(source))
for experiment in tqdm.tqdm(experiments):
    match = re.match(r"^(?!\.).*\s+(\d+)$", experiment)
    if not match:
        continue

    expno, = match.groups()

    # load scorer results from the snapshots
    scores, options = load_experiment(
        os.path.join(source, experiment),
        cache='cache.pk')

    if not options:
        continue

    flat = get_details(options)
    for k, v in flat.items():
        if k not in ignore:
            grid[k].add(v)

    scores = pd.DataFrame.from_dict({
        k: v["test"] for k, v in scores.values()
    }, orient='index')

    results.append((int(expno), scores, flat))

# pick all keys which have more than one unique value
#  and drop any nested model spec changes
grid = [k for k, v in grid.items()
        if len(v) > 1 and  "__model__cls" not in k]

In [ ]:
experiments, scores, manifests = zip(*results)

In [ ]:
params = [{k: opt[k] for k in grid} for opt in manifests]
params = pd.DataFrame.from_dict(dict(zip(experiments, params)), orient="index")

scores = [score[["accuracy", "sparsity"]] for score in tqdm.tqdm(scores)]
scores = pd.concat(dict(zip(experiments, scores)), axis=0, names=["expno"])

In [ ]:
df = scores.unstack(-1)
df.columns = df.columns.to_flat_index().map('-'.join)
df = params.join(df).reset_index()

In [ ]:
df = df.replace({
    "model__cls": {
        "<class 'cplxpaper.mnist.models.real.SimpleConvModel'>": "real.SimpleConvModel",
        "<class 'cplxpaper.mnist.models.complex.SimpleConvModel'>": "cplx.SimpleConvModel",
        "<class 'cplxpaper.mnist.models.real.TwoLayerDenseModel'>": "real.TwoLayerDenseModel",
        "<class 'cplxpaper.mnist.models.complex.TwoLayerDenseModel'>": "cplx.TwoLayerDenseModel",
        "<class 'cplxpaper.mnist.models.real.SimpleDenseModel'>": "real.SimpleDenseModel",
        "<class 'cplxpaper.mnist.models.complex.SimpleDenseModel'>": "cplx.SimpleDenseModel",
    },
    "features__cls": {
        "<class 'cplxpaper.auto.feeds.FeedRawFeatures'>": 'raw',
        "<class 'cplxpaper.auto.feeds.FeedFourierFeatures'>": 'fourier'
    },
})

In [ ]:
df = df.set_index([
    "features__cls", "model__cls", "stages__sparsify__objective__kl_div", "index"
], append=False, drop=True)

In [ ]:
df = df.sort_index(0)

In [ ]:
summary = {}
for k, g in df.groupby(axis=0, level=[0, 1]):
    g = g.loc[k]
    base = g["accuracy-dense"]
    curve = g[["sparsity-sparsify", "accuracy-fine-tune"]]

    curve = curve.mean(level=0).to_numpy()    
    order = curve[:, 0].argsort()

    summary[k] = (base.mean(), base.std()), curve[order]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))

for name, (dense, curve) in summary.items():
    m, s = dense
#     pts = ax.scatter(*curve.T, label=name, s=25)
#     color = pts.get_facecolor()[0]
    spr, acc = curve.T
    pts, = ax.semilogy(1-spr, 1-acc, label=name)
    color = pts.get_color()

    ax.axhspan((1-m)-1.96*s, (1-m)+1.96*s, alpha=0.1, color=color)

#     ax.set_yscale("log")
#     ax.set_xscale("log");# ax.set_xlim(0.01, 1.5)
#     ax.set_xlim(-0.05, 1.05)

ax.legend(ncol=2)
ax.set_title("MNIST")
ax.set_ylabel("error")
ax.set_xlabel("compression")

<br>